## Install YOLO-NAS


In [1]:
!pip install -q git+https://github.com/Deci-AI/super-gradients.git@stable
!pip install -q roboflow
!pip install -q supervision

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.2 requires cubinlinker, which is not installed.
cudf 24.8.2 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.2 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.2 requires cupy-cuda11x>=12.0.0, which is not installed.
ucxx 0.39.1 requires libucx>=1.15.0, which is not installed.
aiobotocore 2.13.2 requires botocore<1.34.132,>=1.34.70, but you have botocore 1.29.165 which is incompatible.
albucore 0.0.13 requires numpy<2,>=1.24.4, but you have numpy 1.23.0 which is incompatible.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


In [3]:
import torch

DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"
MODEL_ARCH = 'yolo_nas_l'

In [4]:
from super_gradients.training import models

model = models.get(MODEL_ARCH, pretrained_weights="coco").to(DEVICE)

The console stream is logged into /root/sg_logs/console.log


[2024-09-24 05:39:10] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2024-09-24 05:39:27] INFO - utils.py - NumExpr defaulting to 4 threads.


[WARNING]No module named 'pycocotools'


[2024-09-24 05:39:31] INFO - env_sanity_check.py - Library check is not supported when super_gradients installed through "git+https://github.com/..." command
[2024-09-24 05:39:32] WARNING - checkpoint_utils.py - :warning: The pre-trained models provided by SuperGradients may have their own licenses or terms and conditions derived from the dataset used for pre-training.
 It is your responsibility to determine whether you have permission to use the models for your use case.
 The model you have requested was pre-trained on the coco dataset, published under the following terms: https://cocodataset.org/#termsofuse
[2024-09-24 05:39:32] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
Downloading: "https://sghub.deci.ai/models/yolo_nas_l

In [5]:
%cd {HOME}

!pip install roboflow

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="OX5LGF0zNUWp6nV9jsTi")
project = rf.workspace("husseinkhan").project("fighter-jet2")
version = project.version(1)
dataset = version.download("yolov5")

/kaggle/working


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Fighter-Jet2-1 in yolov5pytorch::   9%|▉         | 683/7696 [00:00<00:01, 6823.74it/s]

Extracting Dataset Version Zip to Fighter-Jet2-1 in yolov5pytorch::  78%|███████▊  | 6006/7696 [00:00<00:00, 8426.93it/s]

In [6]:
LOCATION = dataset.location
print("location:", LOCATION)
CLASSES = sorted(project.classes.keys())
print("classes:", CLASSES)

In [7]:
MODEL_ARCH = 'yolo_nas_l'
BATCH_SIZE = 8
MAX_EPOCHS = 25
CHECKPOINT_DIR = f'{HOME}/checkpoints'
EXPERIMENT_NAME = project.name.lower().replace(" ", "_")

In [8]:
from super_gradients.training import Trainer

trainer = Trainer(experiment_name=EXPERIMENT_NAME, ckpt_root_dir=CHECKPOINT_DIR)

In [9]:
dataset_params = {
    'data_dir': LOCATION,
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'valid/images',
    'val_labels_dir':'valid/labels',
    'test_images_dir':'test/images',
    'test_labels_dir':'test/labels',
    'classes': CLASSES
}

In [10]:
from super_gradients.training.dataloaders.dataloaders import (
    coco_detection_yolo_format_train, coco_detection_yolo_format_val)

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)

[2024-09-24 05:40:09] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations: 100%|██████████| 2689/2689 [00:00<00:00, 5473.53it/s]
[2024-09-24 05:40:10] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations: 100%|██████████| 769/769 [00:00<00:00, 5584.89it/s]
[2024-09-24 05:40:10] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.


**NOTE:** Let's inspect our dataset

In [11]:
train_data.dataset.transforms

### Instantiate the model

In [12]:
from super_gradients.training import models

model = models.get(
    MODEL_ARCH,
    num_classes=len(dataset_params['classes']),
    pretrained_weights="coco"
)

[2024-09-24 05:40:11] WARNING - checkpoint_utils.py - :warning: The pre-trained models provided by SuperGradients may have their own licenses or terms and conditions derived from the dataset used for pre-training.
 It is your responsibility to determine whether you have permission to use the models for your use case.
 The model you have requested was pre-trained on the coco dataset, published under the following terms: https://cocodataset.org/#termsofuse
[2024-09-24 05:40:11] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2024-09-24 05:40:11] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_l


### Define metrics and training parameters

In [13]:
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

train_params = {
    'silent_mode': False,
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    "max_epochs": MAX_EPOCHS,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

### Train the model

In [14]:
trainer.train(
    model=model,
    training_params=train_params,
    train_loader=train_data,
    valid_loader=val_data
)

[2024-09-24 05:40:12] INFO - sg_trainer.py - Starting a new run with `run_id=RUN_20240924_054012_306107`
[2024-09-24 05:40:12] INFO - sg_trainer.py - Checkpoints directory: /kaggle/working/checkpoints/fighter_jet2/RUN_20240924_054012_306107
[2024-09-24 05:40:12] INFO - sg_trainer.py - Using EMA with params {'decay': 0.9, 'decay_type': 'threshold'}


The console stream is now moved to /kaggle/working/checkpoints/fighter_jet2/RUN_20240924_054012_306107/console_Sep24_05_40_12.txt


Object name `linear_epoch_step` is now deprecated. Please replace it with `LinearEpochLRWarmup`.
initialize_param_groups and update_param_groups usages are deprecated since 3.4.0, will be removed in 3.5.0 and have no effect. 
 Assign different learning rates by passing a mapping of layer name prefixes to lr values through initial_lr training hyperparameter (i.e initial_lr={'backbone': 0.01, 'default':0.1})
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
[2024-09-24 05:40:14] INFO - sg_trainer_utils.py - TRAINING PARAMETERS:
    - Mode:                         Single GPU
    - Number of GPUs:               1          (2 available on the machine)
    - Full dataset size:            2675       (len(train_set))
    - Batch size per GPU:           8          (batch_size)
    - Batch Accumula

SUMMARY OF EPOCH 0
├── Train
│   ├── Ppyoloeloss/loss_cls = 2.262
│   ├── Ppyoloeloss/loss_iou = 0.6355
│   ├── Ppyoloeloss/loss_dfl = 0.6216
│   └── Ppyoloeloss/loss = 3.5191
└── Validation
    ├── Ppyoloeloss/loss_cls = 2.4634
    ├── Ppyoloeloss/loss_iou = 0.425
    ├── Ppyoloeloss/loss_dfl = 0.4923
    ├── Ppyoloeloss/loss = 3.3806
    ├── Precision@0.50 = 0.0076
    ├── Recall@0.50 = 0.0009
    ├── Map@0.50 = 0.0066
    ├── F1@0.50 = 0.0016
    └── Best_score_threshold = 0.05



Validating epoch 1: 100%|██████████| 96/96 [00:35<00:00,  2.68it/s]
[2024-09-24 05:48:09] INFO - base_sg_logger.py - Checkpoint saved in /kaggle/working/checkpoints/fighter_jet2/RUN_20240924_054012_306107/ckpt_best.pth
[2024-09-24 05:48:09] INFO - sg_trainer.py - Best checkpoint overriden: validation mAP@0.50: 0.41626977920532227


...

Validating epoch 23: 100%|██████████| 96/96 [00:36<00:00,  2.67it/s]
[2024-09-24 07:20:14] INFO - base_sg_logger.py - Checkpoint saved in /kaggle/working/checkpoints/fighter_jet2/RUN_20240924_054012_306107/ckpt_best.pth
[2024-09-24 07:20:14] INFO - sg_trainer.py - Best checkpoint overriden: validation mAP@0.50: 0.9691999554634094


SUMMARY OF EPOCH 23
├── Train
│   ├── Ppyoloeloss/loss_cls = 0.6783
│   │   ├── Epoch N-1      = 0.6859 (↘ -0.0076)
│   │   └── Best until now = 0.6859 (↘ -0.0076)
│   ├── Ppyoloeloss/loss_iou = 0.36
│   │   ├── Epoch N-1      = 0.358  (↗ 0.002)
│   │   └── Best until now = 0.358  (↗ 0.002)
│   ├── Ppyoloeloss/loss_dfl = 0.4058
│   │   ├── Epoch N-1      = 0.4096 (↘ -0.0038)
│   │   └── Best until now = 0.4096 (↘ -0.0038)
│   └── Ppyoloeloss/loss = 1.4442
│       ├── Epoch N-1      = 1.4536 (↘ -0.0094)
│       └── Best until now = 1.4536 (↘ -0.0094)
└── Validation
    ├── Ppyoloeloss/loss_cls = 0.61
    │   ├── Epoch N-1      = 0.609  (↗ 0.0011)
    │   └── Best until now = 0.609  (↗ 0.0011)
    ├── Ppyoloeloss/loss_iou = 0.3099
    │   ├── Epoch N-1      = 0.3102 (↘ -0.0003)
    │   └── Best until now = 0.301  (↗ 0.0089)
    ├── Ppyoloeloss/loss_dfl = 0.3895
    │   ├── Epoch N-1      = 0.3929 (↘ -0.0034)
    │   └── Best until now = 0.3889 (↗ 0.0006)
    ├── Ppyoloeloss/loss = 1.3094

Validating epoch 24: 100%|██████████| 96/96 [00:35<00:00,  2.67it/s]


SUMMARY OF EPOCH 24
├── Train
│   ├── Ppyoloeloss/loss_cls = 0.6732
│   │   ├── Epoch N-1      = 0.6783 (↘ -0.0051)
│   │   └── Best until now = 0.6783 (↘ -0.0051)
│   ├── Ppyoloeloss/loss_iou = 0.3567
│   │   ├── Epoch N-1      = 0.36   (↘ -0.0033)
│   │   └── Best until now = 0.358  (↘ -0.0013)
│   ├── Ppyoloeloss/loss_dfl = 0.4012
│   │   ├── Epoch N-1      = 0.4058 (↘ -0.0046)
│   │   └── Best until now = 0.4058 (↘ -0.0046)
│   └── Ppyoloeloss/loss = 1.4312
│       ├── Epoch N-1      = 1.4442 (↘ -0.013)
│       └── Best until now = 1.4442 (↘ -0.013)
└── Validation
    ├── Ppyoloeloss/loss_cls = 0.5958
    │   ├── Epoch N-1      = 0.61   (↘ -0.0142)
    │   └── Best until now = 0.609  (↘ -0.0132)
    ├── Ppyoloeloss/loss_iou = 0.3105
    │   ├── Epoch N-1      = 0.3099 (↗ 0.0006)
    │   └── Best until now = 0.301  (↗ 0.0096)
    ├── Ppyoloeloss/loss_dfl = 0.3877
    │   ├── Epoch N-1      = 0.3895 (↘ -0.0019)
    │   └── Best until now = 0.3889 (↘ -0.0013)
    ├── Ppyoloeloss/loss 

[2024-09-24 07:24:40] INFO - sg_trainer.py - RUNNING ADDITIONAL TEST ON THE AVERAGED MODEL...
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
Validating epoch 25: 100%|██████████| 96/96 [00:36

### Analyze training metrics


In [15]:
# %load_ext tensorboard
# %tensorboard --logdir {CHECKPOINT_DIR}/{EXPERIMENT_NAME}

## Zip and download fine-tuned model

In [16]:
!zip -r yolo_nas.zip {CHECKPOINT_DIR}/{EXPERIMENT_NAME}

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.


  adding: kaggle/working/checkpoints/fighter_jet2/ (stored 0%)
  adding: kaggle/working/checkpoints/fighter_jet2/RUN_20240924_054012_306107/ (stored 0%)
  adding: kaggle/working/checkpoints/fighter_jet2/RUN_20240924_054012_306107/ckpt_latest.pth (deflated 7%)
  adding: kaggle/working/checkpoints/fighter_jet2/RUN_20240924_054012_306107/console_Sep24_05_40_12.txt (deflated 95%)
  adding: kaggle/working/checkpoints/fighter_jet2/RUN_20240924_054012_306107/ckpt_best.pth (deflated 7%)
  adding: kaggle/working/checkpoints/fighter_jet2/RUN_20240924_054012_306107/logs_Sep24_05_40_12.txt (deflated 81%)
  adding: kaggle/working/checkpoints/fighter_jet2/RUN_20240924_054012_306107/average_model.pth

In [17]:
# if you experience 'NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968' error, run code below 👇

# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

## Load trained model

In [18]:
# best_model = models.get(
#     MODEL_ARCH,
#     num_classes=len(dataset_params['classes']),
#     checkpoint_path=f"/kaggle/input/yolo-nas2/pytorch/default/1/ckpt_best.pth"
# ).to(DEVICE)

In [19]:
# # Import the YouTubeVideo class from IPython.display
# from IPython.display import YouTubeVideo

# # Define the YouTube video ID
# video_id = 'l-oo9DJfH_k'  # Replace YOUR_VIDEO_ID with the actual video ID

# # Create a YouTubeVideo object with the specified video ID
# video = YouTubeVideo(video_id)

# # Display the video
# display(video)

In [20]:
# %%capture

# # Define the URL of the YouTube video
# video_url = f'https://www.youtube.com/watch?v={video_id}'

# # Download the video in mp4 format
# !pip install -U "git+https://github.com/ytdl-org/youtube-dl.git"
# !python -m youtube_dl -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4' "$video_url"

# # Print a success message
# print('Video downloaded successfully')


In [21]:
# input_video_path = "/kaggle/working/Battlefield 2042 Trailer - F-35 fighter jet scene-l-oo9DJfH_k.mp4"
# output_video_path = "detections.mp4"

In [22]:
# import torch
# device = 'cuda' if torch.cuda.is_available() else "cpu"

In [23]:
# best_model.to(device).predict(input_video_path).save(output_video_path)

## Evaluate trained model

In [24]:
# trainer.test(
#     model=best_model,
#     test_loader=val_data,
#     test_metrics_list=DetectionMetrics_050(
#         score_thres=0.1,
#         top_k_predictions=300,
#         num_cls=len(dataset_params['classes']),
#         normalize_targets=True,
#         post_prediction_callback=PPYoloEPostPredictionCallback(
#             score_threshold=0.01,
#             nms_top_k=1000,
#             max_predictions=300,
#             nms_threshold=0.7
#         )
#     )
# )

### Calculate confusion matrix

In [25]:
# !pip install onemetric

In [26]:
# import supervision as sv

# ds = sv.DetectionDataset.from_yolo(
#     images_directory_path=f"{dataset.location}/valid/images",
#     annotations_directory_path=f"{dataset.location}/valid/labels",
#     data_yaml_path=f"{dataset.location}/data.yaml",
#     force_masks=False
# )

In [27]:
# import supervision as sv

# CONFIDENCE_TRESHOLD = 0.5

# predictions = {}

# for image_name, image in ds.images.items():
#     result = best_model.predict(image, conf=CONFIDENCE_TRESHOLD)
#     detections = sv.Detections(
#         xyxy=result.prediction.bboxes_xyxy,
#         confidence=result.prediction.confidence,
#         class_id=result.prediction.labels.astype(int)
#     )
#     predictions[image_name] = detections

In [28]:
# import os

# import numpy as np

# from onemetric.cv.object_detection import ConfusionMatrix

# keys = list(ds.images.keys())

# annotation_batches, prediction_batches = [], []

# for key in keys:
#     annotation=ds.annotations[key]
#     annotation_batch = np.column_stack((
#         annotation.xyxy,
#         annotation.class_id
#     ))
#     annotation_batches.append(annotation_batch)

#     prediction=predictions[key]
#     prediction_batch = np.column_stack((
#         prediction.xyxy,
#         prediction.class_id,
#         prediction.confidence
#     ))
#     prediction_batches.append(prediction_batch)

# confusion_matrix = ConfusionMatrix.from_detections(
#     true_batches=annotation_batches,
#     detection_batches=prediction_batches,
#     num_classes=len(ds.classes),
#     conf_threshold=CONFIDENCE_TRESHOLD
# )

# confusion_matrix.plot(os.path.join(HOME, "confusion_matrix.png"), class_names=ds.classes)